In [3]:
import itertools
from pprint import pprint
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import copy
import time
from scipy import special
import matplotlib
import matplotlib.pyplot as plt

In [4]:
from multisoc.infer import data_loader
from multisoc.infer import aux_functions
from multisoc.infer import inference

# Full computation of AddHealth

In [3]:
bad_comms = [48, 1]

### Dictionary to store the results

In [4]:
dimensions_list = ['grade','race','sex']
num_dimensions = len(dimensions_list)
sex_list = ["Female","Male"]
race_list = ["White","Black","Hispanic","Asian","Mixed/other"]
grade_list = ["7th","8th","9th","10th","11th","12th"]
all_attributes_dict = {
    "grade":grade_list,
    "race":race_list,
    "sex":sex_list
}

multidim_groups = list(itertools.product(*[all_attributes_dict[d] for d in dimensions_list]))

In [5]:
aggregation_functions = ["and",
                         "or",
                         "mean"
                        ]
preference_structures = ["multi-1d","1d-full","1d-simple"]

aggr_pref_combinations = [
    "multi-full",
    "and_1d-simple",
    "or_1d-simple",
    "mean_1d-simple",
    "and_multi-1d",
    "and_1d-full"
]

In [6]:
def init_results_dictionary(
    all_attributes_dict,
    aggr_pref_combinations
    ):
    results_dictionary = {
        "school":[], ## School ID
        "N":[] ## Total size
    }

    ## Sizes of one-dimensional groups
    results_dictionary.update(
        {"N_"+i:[] for i in grade_list+race_list+sex_list}
    )

    ## Sizes of multidimensional groups
    multidim_groups = list(itertools.product(*all_attributes_dict.values()))
    results_dictionary.update(
        {"N_"+"|".join(i):[] for i in multidim_groups}
    )

    ## Estimated H for each pair of multidimensional groups
    multidim_pairs = list(itertools.product(multidim_groups,multidim_groups))
    for aggr_pref in aggr_pref_combinations:
        results_dictionary.update(
            {"H_" + aggr_pref + "_" + "|".join(i[0]) + "-" + "|".join(i[1]):[] for i in multidim_pairs}
        )


    ## Estimated h for each pair of one-dimensional groups
    ## Also MRQAP p-values
    ## Also MRQAP mean
    ## Also MRQAP std
    for aggr_pref in aggr_pref_combinations:

        if "1d-simple" in aggr_pref:
            for d, attr_list_d in all_attributes_dict.items():
                onedim_pairs_d = list(itertools.product(attr_list_d,attr_list_d))
                results_dictionary.update(
                    {"h_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                )
                results_dictionary.update(
                    {"h_norm_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                )
                
                if aggr_pref == "and_1d-simple":
                    results_dictionary.update(
                        {"MRQAP_pval1s_h_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                    )
                    results_dictionary.update(
                        {"MRQAP_pval2s_h_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                    )
                    results_dictionary.update(
                        {"MRQAP_av_h_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                    )
                    results_dictionary.update(
                        {"MRQAP_std_h_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                    )
                    results_dictionary.update(
                        {"MRQAP_av_h_norm_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                    )
                    results_dictionary.update(
                        {"MRQAP_std_h_norm_" + aggr_pref + "_" + d + "_" + "-".join(i):[] for i in onedim_pairs_d}
                    )

        elif "1d-full" in aggr_pref:
            onedim_pairs_d = list(itertools.product(itertools.chain.from_iterable(all_attributes_dict.values()),
                                               itertools.chain.from_iterable(all_attributes_dict.values())
                                              ))
            results_dictionary.update(
                {"h_" + aggr_pref + "_" + "-".join(i):[] for i in onedim_pairs_d}
            )
#             results_dictionary.update(
#                 {"MRQAP_pval_" + aggr_pref + "_" + "-".join(i):[] for i in onedim_pairs_d}
#             )
#             results_dictionary.update(
#                 {"MRQAP_av_" + aggr_pref + "_" + "-".join(i):[] for i in onedim_pairs_d}
#             )
#             results_dictionary.update(
#                 {"MRQAP_std_" + aggr_pref + "_" + "-".join(i):[] for i in onedim_pairs_d}
#             )

        elif "multi-1d" in aggr_pref:
            for multidim_group_i in multidim_groups:
                onedim_groups_all = itertools.chain.from_iterable(all_attributes_dict.values())
                results_dictionary.update(
                    {"h_" + aggr_pref + "_" + "|".join(multidim_group_i) + "-" + i:[] for i in onedim_groups_all}
                )
#                 results_dictionary.update(
#                     {"MRQAP_pval_" + aggr_pref + "_" + "|".join(multidim_group_i) + "-" + i:[] for i in onedim_groups_all}
#                 )
#                 results_dictionary.update(
#                     {"MRQAP_av_" + aggr_pref + "_" + "|".join(multidim_group_i) + "-" + i:[] for i in onedim_groups_all}
#                 )
#                 results_dictionary.update(
#                     {"MRQAP_std_" + aggr_pref + "_" + "|".join(multidim_group_i) + "-" + i:[] for i in onedim_groups_all}
#                 )

    ## Likelihood, AIC, BIC
    results_dictionary.update({"L_"+i:[] for i in aggr_pref_combinations})
    results_dictionary.update({"AIC_"+i:[] for i in aggr_pref_combinations})
    results_dictionary.update({"BIC_"+i:[] for i in aggr_pref_combinations})
    
    return results_dictionary

### Functions to introduce data in dictionary

In [7]:
def save_group_sizes(n,results_dictionary,multidim_groups,all_attributes_dict):
    for col in multidim_groups:
        if col in n.columns:
            results_dictionary["N_"+"|".join(col)].append(n[col]["N"])
        else:
            results_dictionary["N_"+"|".join(col)].append(np.nan)
    for d, attr_lst in all_attributes_dict.items():
        aggr_counts = n.transpose().groupby(d).sum()
        for i in attr_lst:
            if i in aggr_counts["N"]:
                results_dictionary["N_"+i].append(aggr_counts["N"][i])
            else:
                results_dictionary["N_"+i].append(np.nan)
    return results_dictionary

def save_multidim_preferences(
    aggr_fun,
    H,
    multidim_groups,
    results_dictionary
    ):
    for col1 in multidim_groups:
        for col2 in multidim_groups:
            ## To extract info from a pandas dataframe it goes like df[column][row] instead of [row][column] like an array
            if col2 in H.columns and col1 in H.index:
                results_dictionary["H_" + aggr_fun + "_" + "|".join(col1) + "-" + "|".join(col2)].append(H[col2][col1])
            else:
                results_dictionary["H_" + aggr_fun + "_" + "|".join(col1) + "-" + "|".join(col2)].append(np.nan)
    return results_dictionary

def save_likelihoods(
    aggr_pref,
    likel,
    results_dictionary
    ):
    results_dictionary["L_"+aggr_pref].append(likel[0])
    results_dictionary["AIC_"+aggr_pref].append(likel[1])
    results_dictionary["BIC_"+aggr_pref].append(likel[2])
    return results_dictionary
    
def save_1d_simple_preferences(
    aggr_fun,
    dimensions_list,
    h_est_simple,
    results_dictionary,
    all_attributes_dict,
    name_prefix = "h_" ## h_norm_
    ):
    for i, d in enumerate(dimensions_list):
        attr_lst_d = all_attributes_dict[d]
        onedim_pairs_d = itertools.product(attr_lst_d,attr_lst_d) 
        for onedim_pair_i in onedim_pairs_d:
            try:
                h_est_simple[i][onedim_pair_i[1]][onedim_pair_i[0]]
            except KeyError:
                results_dictionary[name_prefix + aggr_fun + "_1d-simple_" + d + "_" + "-".join(onedim_pair_i)].append(np.nan)
                continue
            ## To extract info from a pandas dataframe it goes like df[column][row] instead of [row][column] like an array
            results_dictionary[name_prefix + aggr_fun + "_1d-simple_" + d + "_" + "-".join(onedim_pair_i)].append(h_est_simple[i][onedim_pair_i[1]][onedim_pair_i[0]])
    return results_dictionary

def save_multi_1d_preferences(
    dimensions_list,
    h_est_multi_1d,
    results_dictionary,
    all_attributes_dict,
    aggr_fun = "and"
    ):
    for i, d in enumerate(dimensions_list):
        attr_list = all_attributes_dict[d]
        ## Assert that there are some non-nan data there (and/or that I'm correctly aligning the matrices from h_est_multi_1d with my lists of attributes)
        assert set(h_est_multi_1d[i].columns).intersection(set(attr_list))
        assert set(h_est_multi_1d[i].index).intersection(set(multidim_groups))
        for multi_group in multidim_groups:
            for onedim_group in attr_list:
                try:
                    val = h_est_multi_1d[i][onedim_group][multi_group]
                except KeyError:
                    val = np.nan
                results_dictionary["h_" + aggr_fun + "_multi-1d_" + "|".join(multi_group) + "-" + onedim_group].append(val)
    return results_dictionary

def save_full_1d_preferences(
    dimensions_list,
    h_est_full_1d,
    results_dictionary,
    all_attributes_dict,
    aggr_fun = "and"
    ):
    for i1, d1 in enumerate(dimensions_list):
        for i2, d2 in enumerate(dimensions_list):
            h_est_d1d2 = h_est_full_1d[(i1,i2)]
            ## Assert that there are some non-nan data there (and/or that I'm correctly aligning the matrices from h_est_full_1d with my lists of attributes)
            assert set(h_est_d1d2.columns).intersection(set(all_attributes_dict[d2]))
            assert set(h_est_d1d2.index).intersection(set(all_attributes_dict[d1]))
            for attr1 in all_attributes_dict[d1]:
                for attr2 in all_attributes_dict[d2]:
                    try:
                        val = h_est_d1d2[attr2][attr1]
                    except KeyError:
                        val = np.nan
                    results_dictionary["h_" + aggr_fun + "_1d-full_" + attr1 + "-" + attr2].append(val)
    return results_dictionary

### Main computations

In [8]:
mrqap_iter = 100
data_path = "../multidimensional_homophily/Datasets/AddHealth" ## Change by the path to the AddHealth dataset
results_path = "."

In [ ]:
results_dictionary = init_results_dictionary(all_attributes_dict,aggr_pref_combinations)

for school in range(1,8): ## For tests
# for school in range(1,85): ## Full computation
    t0 = time.time()
    if school in bad_comms:
        continue
#     try:
    nodes_list,edges_list = data_loader.load_AddHealth(school, 
                                                       th_sex = 20, 
                                                       th_race= 20, 
                                                       th_grade= 20, 
                                                       school= None, 
                                                       remove_unreported=True,
                                                       data_path = data_path
                                                      )
#     except: print('Error')
    if len(nodes_list['sex'].cat.categories)==1 or len(nodes_list['race'].cat.categories)==1:
        print('There is only one sex category or only one race category: we skip it')
    elif nodes_list.shape[0]<100:
        print('There are less than 100 students in this school: we skip it')
    else:
        ## Store school number and size
        results_dictionary["school"].append(school)
        results_dictionary["N"].append(nodes_list.shape[0])
        
        ## Compute group sizes
        n0,counts0 = aux_functions.get_n_and_counts(nodes_list,edges_list,dimensions_list)
        ## To ensure CONSISTENT ORDERING
        ## First extract only the columns that exist in the dataframe, otherwise we 
        ## run into problems because, even if we drop NaNs, the resulting dataframe, 
        ## internally, thinks that there are more nonempty columns than there really are
        new_cols = [i for i in multidim_groups if i in n0.columns] 
        n = n0.reindex(columns=new_cols)#.dropna(how="all",axis=1) ## If we use this instead of "new_cols" we end up with empty columns that change n.columns.levshape which is used later for many computations (g_vec=n.columns.levshape)
        
        new_index = [i for i in multidim_groups if i in counts0.index]
        new_cols = [i for i in multidim_groups if i in counts0.columns]
        counts = counts0.reindex(index=new_index,columns=new_cols       ## Change order of columns and rows
                                )#.dropna(how="all",axis=0).dropna(how="all",axis=1) ## Remove empty rows and columns
        
        ## Compute 1D group sizes and inter-group links
        att_pop0 = [n.T.groupby(level=i, sort=False).sum() for i in range(num_dimensions)]
        att_counts0 = [counts.T.groupby(level=i,sort=False).sum().T.groupby(level=i, sort=False).sum() for i in range(num_dimensions)]
        ## To ensure consistent ordering
        att_counts = []
        for i, cnts in enumerate(att_counts0):
            assert set(all_attributes_dict[dimensions_list[i]]).intersection(set(cnts.index))
            assert set(all_attributes_dict[dimensions_list[i]]).intersection(set(cnts.columns))
            new_index = [attr_i for attr_i in all_attributes_dict[dimensions_list[i]] if attr_i in cnts.index]
            new_cols = [attr_i for attr_i in all_attributes_dict[dimensions_list[i]] if attr_i in cnts.columns]
            att_counts.append( cnts.reindex(index=new_index, columns=new_cols) )
        att_pop = []
        for i, cnts in enumerate(att_pop0):
            assert set(all_attributes_dict[dimensions_list[i]]).intersection(set(cnts.index))
            new_index = [attr_i for attr_i in all_attributes_dict[dimensions_list[i]] if attr_i in cnts.index]
            att_counts.append( cnts.reindex(index=new_index) )
#         att_counts = [ cnts.reindex(index=all_attributes_dict[dimensions_list[i]],columns=all_attributes_dict[dimensions_list[i]]
#                                    ).dropna(how="all",axis=1).dropna(how="all",axis=0) 
#                       for i,cnts in enumerate(att_counts0)]
#         att_pop = [ cnts.reindex(index=all_attributes_dict[dimensions_list[i]]
#                                    ).dropna(how="all",axis=0) 
#                       for i,cnts in enumerate(att_pop0)]
        
        print(f'There are {n.shape[1]} parameters')
        results_dictionary = save_group_sizes(n,results_dictionary,multidim_groups,all_attributes_dict)

        g_vec = n.columns.levshape

        ################################################
        ## Multidimensional
        print ("----------------------\nMultidimensional")
        H_est_multi, _, num_params = inference.estimate_H(n,counts,type_p='multidimensional',print_convergence=True)
        likel_multi = inference.compute_likel(n,counts,H_est_multi,k=num_params,print_values=False)
        
        results_dictionary = save_multidim_preferences(
            "multi-full",
            H_est_multi,
            multidim_groups,
            results_dictionary
            )
        results_dictionary = save_likelihoods(
            "multi-full",
            likel_multi,
            results_dictionary
            )
        
        ################################################
        ## Simple
        for aggr_fun in aggregation_functions:
            print ("----------------------\nSimple ", aggr_fun)
            H_est_simple, x_est, num_params = inference.estimate_H(n,counts,type_p=aggr_fun,print_convergence=True)
            num_free_params = aux_functions.product_mean_free_params(g_vec)
            likel_simple = inference.compute_likel(n,counts,H_est_simple,k=num_free_params,print_values=False)
            
            ## Store "and" result for later for MRQAP
            if aggr_fun == "and":
                x_est_and = copy.deepcopy(x_est)

            results_dictionary = save_multidim_preferences(
                aggr_fun + "_1d-simple",
                H_est_simple,
                multidim_groups,
                results_dictionary
                )
            results_dictionary = save_likelihoods(
                aggr_fun + "_1d-simple",
                likel_simple,
                results_dictionary
                )
            
            ## Extract and save 1D preferences
            h_est_mtrx = aux_functions.vec_to_mat_list(x_est,g_vec)
            h_est_simple = []
            h_est_norm = []
            for i in range(num_dimensions):
                h_est_simple.append(pd.DataFrame(h_est_mtrx[i],index=att_counts[i].index,columns=att_counts[i].columns))
                h_est_norm.append(h_est_simple[i].div(h_est_simple[i].to_numpy().diagonal(), axis=0))
            save_1d_simple_preferences(
                aggr_fun,
                dimensions_list,
                h_est_simple,
                results_dictionary,
                all_attributes_dict,
                name_prefix = "h_" ## h_norm
                )
            save_1d_simple_preferences(
                aggr_fun,
                dimensions_list,
                h_est_norm,
                results_dictionary,
                all_attributes_dict,
                name_prefix = "h_norm_"
                )
        
        ################################################
        ## Multi -> 1D
        print ("----------------------\nMulti->1D")
        H_est_multi_1d, x_est, num_params = inference.estimate_H(n,counts,
                                                                 type_p="product_weights",
                                                                 print_convergence=True,
                                                                opt_options = {'ftol':1e-10,'gtol':1e-10,'maxfun':1000000})
        num_free_params = aux_functions.multi_1d_free_params(g_vec)
        likel_multi_1d = inference.compute_likel(n,counts,H_est_multi_1d,k=num_free_params,print_values=False)
        print(likel_multi_1d)
        
        results_dictionary = save_multidim_preferences(
                "and_multi-1d",
                H_est_multi_1d,
                multidim_groups,
                results_dictionary
                )
        results_dictionary = save_likelihoods(
                "and_multi-1d",
                likel_multi_1d,
                results_dictionary
                )
        
        ## Extract and save multi->1D preferences
        h_est_mtrx = aux_functions.vec_to_weights_matrix(x_est,g_vec)
        h_est_multi_1d = []
        for d in range(num_dimensions):
            h_est_multi_1d.append(pd.DataFrame(h_est_mtrx[d],index=H_est_multi_1d.index,columns=att_counts[d].columns))

        results_dictionary = save_multi_1d_preferences(
            dimensions_list,
            h_est_multi_1d,
            results_dictionary,
            all_attributes_dict,
            aggr_fun = "and"
            )
        
        ################################################
        ## Full 1D
        print ("----------------------\nFull 1D")
        H_est_full_1d, x_est, num_params = inference.estimate_H(n,counts,
                                                                type_p="product_cross_dimensional",
                                                                print_convergence=True,
                                                               opt_options = {'ftol':1e-10,'gtol':1e-10,'maxfun':1000000})
        num_free_params = aux_functions.full_1d_free_params(g_vec)
        likel_full_1d = inference.compute_likel(n,counts,H_est_full_1d,k=num_free_params,print_values=False)
        print(likel_full_1d)
        
        results_dictionary = save_multidim_preferences(
                "and_1d-full",
                H_est_full_1d,
                multidim_groups,
                results_dictionary
                )
        results_dictionary = save_likelihoods(
                "and_1d-full",
                likel_full_1d,
                results_dictionary
                )
        
        ## Extract and save full 1D preferences
        h_est_mtrx = aux_functions.vec_to_mat_dict_cross_one_dimensional(x_est,g_vec)
        h_est_full_1d = {}
        for d1 in range(num_dimensions):
            for d2 in range(num_dimensions):
                h_est_full_1d[(d1,d2)] = pd.DataFrame(h_est_mtrx[(d1,d2)],index=att_counts[d1].index,columns=att_counts[d2].columns)
            
        results_dictionary = save_full_1d_preferences(
            dimensions_list,
            h_est_full_1d,
            results_dictionary,
            all_attributes_dict,
            aggr_fun = "and"
            )
        
        ################################################
        ## MRQAP "and simple"
        print ("MRQAP")
        
#         num_params = np.sum(g_vec)**2
        num_params = len(x_est_and)
        X_rnd = np.zeros((mrqap_iter,num_params))
        X_rnd_norm = np.zeros((mrqap_iter,num_params))
        
        ## Initialize node list copy to randomize
        nodes_list_rnd = copy.deepcopy(nodes_list)
        
        for it in tqdm(range(mrqap_iter)):
            
            ## Randomize nodes order
            nodes_list_rnd[:] = nodes_list.sample(frac=1,replace=False).values
            
            ## Count inter-group links
            n_rnd0,counts_rnd0 = aux_functions.get_n_and_counts(nodes_list_rnd,edges_list,dimensions_list)
            ## To ensure consistent ordering
            new_cols = [i for i in multidim_groups if i in n_rnd0.columns] 
            n_rnd = n_rnd0.reindex(columns=new_cols)
            
            new_index = [i for i in multidim_groups if i in counts_rnd0.index]
            new_cols = [i for i in multidim_groups if i in counts_rnd0.columns]
            counts_rnd = counts_rnd0.reindex(index=new_index,columns=new_cols)
    
            ## Infer preferences
            _, x_rnd, _ = inference.estimate_H(n_rnd,counts_rnd,type_p="and",print_convergence=True)
            X_rnd[it,:] = x_rnd
            
            ## Normalize x_rnd
            h_mtrx_rnd = aux_functions.vec_to_mat_list(x_rnd,g_vec)
            h_mtrx_rnd_norm = [np.divide(i.T, np.diag(i)).T for i in h_mtrx_rnd]
            x_rnd_norm = np.array(list(itertools.chain.from_iterable([i.ravel() for i in h_mtrx_rnd_norm])))
            X_rnd_norm[it,:] = x_rnd_norm
            
        X_av = np.mean(X_rnd,axis=0)
        X_std = np.std(X_rnd,axis=0)
        X_norm_av = np.mean(X_rnd_norm,axis=0)
        X_norm_std = np.std(X_rnd_norm,axis=0)
        
        ## Normalized version of empirical preferences
        h_mtrx_est_and = aux_functions.vec_to_mat_list(x_est_and,g_vec)
        h_mtrx_est_and_norm = [np.divide(i.T, np.diag(i)).T for i in h_mtrx_est_and]
        x_est_and_norm = np.array(list(itertools.chain.from_iterable([i.ravel() for i in h_mtrx_est_and_norm])))

        ## Compute p-value
        X_pval_oneside = np.sum(np.less_equal(X_rnd_norm,x_est_and_norm),axis=0) / X_rnd.shape[0] ## Proportion of iterations where preference is less than empirical
        assert np.all(X_pval_oneside>=0) and np.all(X_pval_oneside<=1) ## To avoid further mistakes
        X_pval_twoside = copy.deepcopy(X_pval_oneside)
        X_pval_twoside[X_pval_twoside>0.5] = 1-X_pval_twoside[X_pval_twoside>0.5]
        ## Careful, this is not the two-sided, which would be 2 times that value if the distribution is symmetric
        
        data_and_labels = [
            (X_av,"MRQAP_av_h_"),
            (X_std,"MRQAP_std_h_"),
            (X_norm_av,"MRQAP_av_h_norm_"),
            (X_norm_std,"MRQAP_std_h_norm_"),
            (X_pval_oneside,"MRQAP_pval1s_h_"),
            (X_pval_twoside,"MRQAP_pval2s_h_")
            ]
        
        ## Extract and save 1D average preferences
        att_counts_rnd0 = [counts_rnd.T.groupby(level=i,sort=False).sum().T.groupby(level=i, sort=False).sum() for i in range(num_dimensions)]
        ## To ensure consistent ordering
        att_counts_rnd = []
        for i, cnts in enumerate(att_counts_rnd0):
            assert set(all_attributes_dict[dimensions_list[i]]).intersection(set(cnts.index))
            assert set(all_attributes_dict[dimensions_list[i]]).intersection(set(cnts.columns))
            new_index = [attr_i for attr_i in all_attributes_dict[dimensions_list[i]] if attr_i in cnts.index]
            new_cols = [attr_i for attr_i in all_attributes_dict[dimensions_list[i]] if attr_i in cnts.columns]
            att_counts_rnd.append( cnts.reindex(index=new_index, columns=new_cols) )
        
        for x_est, name_prefix in data_and_labels:
            h_est_mtrx = aux_functions.vec_to_mat_list(x_est,g_vec)
            h_est_simple = []
            for i in range(num_dimensions):
                h_est_simple.append(pd.DataFrame(h_est_mtrx[i],index=att_counts_rnd[i].index,columns=att_counts_rnd[i].columns))

            save_1d_simple_preferences(
                    "and",
                    dimensions_list,
                    h_est_simple,
                    results_dictionary,
                    all_attributes_dict,
                    name_prefix = name_prefix
                    )
    
        ## Save each iteration to ensure that even if there is an error we get some data
        results_df = pd.DataFrame(results_dictionary)
        results_df.to_csv(results_path+"/paper_results.csv")
        ## v2 -- introduced num_free_params, which gives the correct AIC and BIC for simple, multi-1D, and full-1d
        ## v3 -- corrected MRQAP incorrect normalization X_rnd.shape[0] instead of original X_rnd.shape[1]; I also changed comparison to X_rnd_norm, because due to unidentifiability of product, comparing unnormalized preferences may be unreliable
        print ("Time for full iteration: ", time.time()-t0)
        print ("++++++++++++++++++++++++++++++++++++++++")

Community #2: one school
There is only one sex category or only one race category: we skip it
Community #3: one school
There are less than 100 students in this school: we skip it
Community #4: 2 schools
There is only one sex category or only one race category: we skip it
Community #5: one school
There is only one sex category or only one race category: we skip it
Community #6: one school
There is only one sex category or only one race category: we skip it
Community #7: 2 schools
There are 36 parameters
----------------------
Multidimensional
----------------------
Simple  and
Likelihood maximization convergence result: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------
Simple  or
Likelihood maximization convergence result: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------
Simple  mean
Likelihood maximization convergence result: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
----------------------
Multi->1D
Likelihood maximization convergence re

### Checking consistency of old and new results

In [ ]:
old_results = pd.read_csv("preference_results_v4_merged.csv")

In [ ]:
results_df[results_df["school"]==7]

In [ ]:
old_results[old_results["school"]==7]

In [ ]:
## New results are almost the exact same but for AIC and BIC of full multidimensional model because now we subtract the 
## number of NaN elements in the multidimensional matrix to be a bit more conservative

## Some of the estimated latent preferences can also be different because the model is not fully specified - in the
## paper we explain that to deal with that issue we normalize preferences. After adequate normalization (e.g. row-wise)
## preferences are the same between different runs.

xx = []
yy = []
for col in results_df.columns:
    print (col, old_results[old_results["school"]==7][col].values[0], results_df[col].values[0])
    xx.append(old_results[old_results["school"]==7][col].values[0])
    yy.append(results_df[col].values[0])

In [ ]:
plt.close("all")
plt.plot(xx,yy,"o")
plt.plot([min(xx),max(xx)],[min(xx),max(xx)],"-k")
plt.show()

#### Example of normalization with multi->1D preference structure

In [ ]:
xx = []
yy = []

for col in results_df.columns:
    if col.startswith("h_and_multi-1d_7th|Hispanic|Female") and col.endswith("ale"):
        print (col, old_results[old_results["school"]==7][col].values[0], results_df[col].values[0])
        xx.append(old_results[old_results["school"]==7][col].values[0])
        yy.append(results_df[col].values[0])
        
xx = np.array(xx)
yy = np.array(yy)

print ("Same values after normalization", xx/xx[~np.isnan(xx)].sum(), yy/yy[~np.isnan(xx)].sum())

In [ ]:
xx = []
yy = []

for col in results_df.columns:
    if col.startswith("h_and_multi-1d_7th|Hispanic|Female") and col.endswith("th"):
        print (col, old_results[old_results["school"]==7][col].values[0], results_df[col].values[0])
        xx.append(old_results[old_results["school"]==7][col].values[0])
        yy.append(results_df[col].values[0])
        
xx = np.array(xx)
yy = np.array(yy)

print ("Same values after normalization", "\n",xx/xx[~np.isnan(xx)].sum(), "\n",yy/yy[~np.isnan(xx)].sum())

In [ ]:
xx = []
yy = []

for col in results_df.columns:
    if col.startswith("h_and_multi-1d_7th|Hispanic|Female") and not col.endswith("th") and not col.endswith("ale"):
        print (col, old_results[old_results["school"]==7][col].values[0], results_df[col].values[0])
        xx.append(old_results[old_results["school"]==7][col].values[0])
        yy.append(results_df[col].values[0])
        
xx = np.array(xx)
yy = np.array(yy)

print ("Same values after normalization", "\n",xx/xx[~np.isnan(xx)].sum(), "\n",yy/yy[~np.isnan(xx)].sum())

#### Check MRQAP values
Some values can be over 1 because in the normalization we divide by the diagonal, so it is natural that the average of MRQAP, after normalization, is around 1 (all values should be quite neutral).

In [ ]:
xx = []
yy = []

for col in results_df.columns:
    # if "MRQAP" in col and "norm" in col and "av" in col:
    if "MRQAP" in col:
        print (col, old_results[old_results["school"]==7][col].values[0], results_df[col].values[0])
        xx.append(old_results[old_results["school"]==7][col].values[0])
        yy.append(results_df[col].values[0])

In [ ]:
plt.close("all")
plt.plot(xx,yy,"o")
plt.plot([0,1.2],[0,1.2],"-k")
plt.show()